# Reto 06: Morfología de los tokens

In [2]:
# Importación dela librería de SpaCy
import spacy

# Cargamos núcleo de trabajo (Español)
pln_es = spacy.load("es_core_news_lg")

In [3]:
# Ejemplo de extracción de etiquetas en frases
Ejemplo = pln_es(u'La locura es como la gravedad, solo se necesita un empujoncito')
for token in Ejemplo:
    print("{:15s} {:s}".format(token.text, str(token.morph)))

La              Definite=Def|Gender=Fem|Number=Sing|PronType=Art
locura          Gender=Fem|Number=Sing
es              Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin
como            
la              Definite=Def|Gender=Fem|Number=Sing|PronType=Art
gravedad        Gender=Fem|Number=Sing
,               PunctType=Comm
solo            
se              Case=Acc|Person=3|PrepCase=Npr|PronType=Prs|Reflex=Yes
necesita        Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin
un              Definite=Ind|Gender=Masc|Number=Sing|PronType=Art
empujoncito     Gender=Masc|Number=Sing


In [4]:
# Ejemplo de extracción de etiquetas en frases
Ejemplo = pln_es(u'Mi perro y mi gato se pelearon ayer porque uno se llevó el juguete del otro')
for token in Ejemplo:
    print("{:10s} {:10s} {:s}".format(token.text, token.pos_, str(token.morph)))

Mi         DET        Number=Sing|Number[psor]=Sing|Person=1|Poss=Yes|PronType=Prs
perro      PROPN      
y          CCONJ      
mi         DET        Number=Sing|Number[psor]=Sing|Person=1|Poss=Yes|PronType=Prs
gato       NOUN       Gender=Masc|Number=Sing
se         PRON       Case=Acc|Person=3|PrepCase=Npr|PronType=Prs|Reflex=Yes
pelearon   VERB       Mood=Ind|Number=Plur|Person=3|Tense=Past|VerbForm=Fin
ayer       ADV        
porque     SCONJ      
uno        PRON       Gender=Masc|Number=Sing|PronType=Ind
se         PRON       Case=Dat|Person=3|PrepCase=Npr|PronType=Prs|Reflex=Yes
llevó      VERB       Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin
el         DET        Definite=Def|Gender=Masc|Number=Sing|PronType=Art
juguete    NOUN       Gender=Masc|Number=Sing
del        ADP        Definite=Def|Gender=Masc|Number=Sing|PronType=Art
otro       PRON       Gender=Masc|Number=Sing|PronType=Ind


In [5]:
# Frases para identificar
Frases = [
    pln_es(u"Todos los niños, jugarán futbol durante el recreo"),
    pln_es(u"La águila, hace su nido sobre la montaña"),
    pln_es(u"En la conveción habían muchas personas"),
    pln_es(u"Los hechos que relata el testigo durante el juicio fueron confirmados"),
    pln_es(u"Todas las personas quedaron asombradas"),
    pln_es(u"La agua, comenzó a caer"),
    pln_es(u"La hacha se confirmó encontrada en la cochera"),
    pln_es(u"Estaban muchas aves volando en el cielo cuando de pronto..."),
    pln_es(u"Había muchas personas en esa ciudad, sin embargo, nadie..."),
    pln_es(u"En un día normal yo trabajo, estudio, y me divierto"),
    pln_es(u"Será necesario que tomemos este asunto muy en serio"),
    pln_es(u"Las cosas que sabes, fueron tomadas de un libro")]

In [6]:
# Módulo para identificar Caso 1 (coma entre sujeto y predicado)
# Tips:
# A) Encontrar en la frase los tokens que sean iguales a ','
# B) Verificar que el token anterior a ese sea un Sujeto (NOUN o PROPN)
# y que el posterior sea un verbo o verbo auxiliar (VERB o AUX)

def Case1(Frase):
    Tokens = [token for token in Frase]
    for token in Tokens:
        if token.text == ",":
            i = Tokens.index(token)
            if ((Tokens[i-1].pos_ == "NOUN" or Tokens[i-1].pos_ == "PROPN") and
               (Tokens[i+1].pos_ == "VERB" or Tokens[i+1].pos_ == "AUX")):
                print(Frase)
    
print("Errores de Caso1 encontrados:\n")
for Frase in Frases:
    Case1(Frase)

Errores de Caso1 encontrados:

Todos los niños, jugarán futbol durante el recreo
La águila, hace su nido sobre la montaña
La agua, comenzó a caer


In [7]:
# Módulo para identificar Caso 2 (Determinantes de palabras que empiezan con a tónica)
# Nota: Identificar la sílaba tónica de una palabra requiere de un proceso
# más complejo, en el ejercicio, identifica sí existe "la" + "á____",
# "la" + "a____" (<=5 caracteres) o "la" + "ha____" (<=5 caracteres)
# Tips:
# A) Encontrar en la frase los tokens que contengan "la" o "La" (Usar lower())
# B) Verificar que el token siguiente tenga en su primer caracter "á" o que
# en sus dos primeros caracteres tenga "a_" o "ha" con menos de 6 caracteres totales

def Case2(Frase):
    Tokens = [token for token in Frase]
    for token in Tokens:
        if token.text.lower() == "la":
            i = Tokens.index(token)
            next_word = Tokens[i + 1].text
            if (next_word[0] == "á" or
                (next_word[0] == "a" and len(next_word) <= 5) or
                (next_word[:2] == "ha" and len(next_word) <=5)):
                 print(Frase)
            

print("Errores de Caso2 encontrados:\n")
for Frase in Frases:
        Case2(Frase)

Errores de Caso2 encontrados:

La águila, hace su nido sobre la montaña
La agua, comenzó a caer
La hacha se confirmó encontrada en la cochera


In [8]:
# Módulo para identificar Caso 3 (Verbo + sujero en plural)
# Nota, la combinación incorrecta a econtrar es:
# Auxiliar(En plural) + Determinante
# (Había mucha gente) (Habían muchas personas)
# Tips:
# A) Encontrar en los tokens, aquellos que sean verbos auxiliares (AUX)
# B) Verificar que ese verbo auxiliar sea plural (Number=Plur)
# C) Verificar que el siguiente PoS corresponda a un determinante (DET)

def Case3(Frase):
    Tokens = [token for token in Frase]
    for token in Tokens:
        if token.pos_ == "AUX" and token.morph.get("Number")[0] == "Plur":
            i = Tokens.index(token)
            if Tokens[i + 1].pos_ == "DET":
                print(Frase)


print("Errores de Caso3 encontrados:\n")
for Frase in Frases:
    Case3(Frase)

Errores de Caso3 encontrados:

En la conveción habían muchas personas


In [9]:
# Módulo para identificar Caso 4 (Mezclar tiempos gramaticales)
# Aplica cuando en la frase, existen dos tiempos gramaticales
# distintos, pero no hay ninguna conjunción (De cualquier tipo)
# que separe la frase en 2.
# Tips:
# A) Encontrar en la frase los pos que correspondan a verbos (VERB)
# B) Ver si dentro de la misma frase existen verbos con diferentes tiempos gramaticales
# C) Revisar si dentro de esa misma frase existe algún tipo de conjunción, coma o punto que
# separe las ideas. (Solo verificar si existen)

def Case4(Frase):
    Tokens = [token for token in Frase]
    tenses = []
    Conj = False

    for token in Tokens:
        if token.pos_ == "VERB" and token.morph.get("Tense") != []:
            tenses.append(token.morph.get("Tense")[0])

        if (token.pos_ == "CONJ" or token.pos_ == "CCONJ" or token.pos_ == "SCONJ"
            or token.text == "," or token.text == "."):
            Conj = True

    if len(set(tenses)) > 1 and Conj == False:
        print(Frase)


print("Errores de Caso4 encontrados:\n")
for Frase in Frases:
    Case4(Frase)

Errores de Caso4 encontrados:

Los hechos que relata el testigo durante el juicio fueron confirmados
